In [7]:
import uproot
import numpy as np
import matplotlib.pylab as plt
import matplotlib.colors as colors
from scipy.optimize import curve_fit
# import file_reading from src file 
from src.file_reading import read_file
from importlib import reload


In [8]:
events_path = "data/example_file.root"

In [9]:
events_sim = uproot.open(events_path)

In [13]:
#reload file_reading
#pT, pX, pY, pZ = read_file(MAX_EVENTS=5000, mode=0)